In [1]:
from mapping_reef_classes import get_coordinates
import os
import re
import pandas as pd
import numpy as np

In [2]:
def list_files(filepath, filetype):
   paths = []
   for root, dirs, files in os.walk(filepath):
      for file in files:
         if file.lower().endswith(filetype.lower()):
            paths.append(os.path.join(root, file))
   return(paths)

def list_collects(filepath):
   paths = []
   for root, dirs, files in os.walk(filepath):
      for dir in dirs:
        paths.append(os.path.join(root, dir))
   collects = [re.findall('.*\d+_\d+_\w+_\w{4}\Z', i) for i in paths]
   collects = list(set([item for sublist in collects for item in sublist]))
   collects.sort()
   return(collects)

In [3]:
tif_files = list_files(r"C:\Users\ageglio\OneDrive - DOI\Documents - Reef Mapping\Data Release", 'tif')
# further sorting of file search
tif_files = [re.findall('.*Classified.*', i) for i in tif_files]
tif_files = list(set([item for sublist in tif_files for item in sublist]))
tif_files.sort()
tif_files

['C:\\Users\\ageglio\\OneDrive - DOI\\Documents - Reef Mapping\\Data Release\\2021 Sites\\Bay Harbor\\BayHarbor_Classified2.tif',
 'C:\\Users\\ageglio\\OneDrive - DOI\\Documents - Reef Mapping\\Data Release\\2021 Sites\\Elk Rapids\\ElkRapids_Classified2.tif',
 'C:\\Users\\ageglio\\OneDrive - DOI\\Documents - Reef Mapping\\Data Release\\2021 Sites\\Fishermens Island\\FishermensIsland_Classified.tif',
 'C:\\Users\\ageglio\\OneDrive - DOI\\Documents - Reef Mapping\\Data Release\\2021 Sites\\Ingalls Point\\IngallsPoint_Classified.tif',
 'C:\\Users\\ageglio\\OneDrive - DOI\\Documents - Reef Mapping\\Data Release\\2021 Sites\\Mission\\MissionPeninsula_Classified_test.tif',
 'C:\\Users\\ageglio\\OneDrive - DOI\\Documents - Reef Mapping\\Data Release\\2021 Sites\\Tannery Creek\\TanneryCreek_Classified.tif',
 'C:\\Users\\ageglio\\OneDrive - DOI\\Documents - Reef Mapping\\Data Release\\2021 Sites\\Traverse Shoal\\TraverseShoal_Classified.tif']

In [4]:
df_tiffs = get_coordinates().return_min_max_tif_df(tif_files)
# df_tiffs.to_csv("reef_coordinates.csv")
df_tiffs.dtypes

filename     object
min_lat     float64
min_lon     float64
max_lat     float64
max_lon     float64
avr_lat     float64
avr_lon     float64
dtype: object

In [5]:
# collects2019 = list_collects(r"W:\2019") + list_collects(r"M:\2019")
# collects2020 = list_collects(r"W:\2020")
# collects2021 = list_collects(r"Z:\NewestCollects\2021") + list_collects(r"W:\2021")
# collects2022 = list_collects(r"Z:\NewestCollects\2022")
# collects2023 = list_collects(r"Z:\NewestCollects\2023") + list_collects(r"X:\AUV Collects\2023")
# all_collects = collects2019 + collects2020 + collects2021 + collects2022 + collects2023
# df_collects_latlon = get_coordinates().return_MissionLog_min_max_df(all_collects)
# df_collects_latlon.to_csv("collects_lat_lon.csv")

df = pd.read_csv("collects_lat_lon.csv", index_col=0)
df['Reef_overlap'] = 'nan'
df

,collect_path,collect_id,date,collect,Iver_num,cam_sys,min_lat,min_lon,max_lat,max_lon,Reef_overlap
0,W:\2019\20191204_001_Iver3069_ABS1,20191204_001_Iver3069_ABS1,20191204,1,Iver3069,ABS1,44.046790,-76.170749,44.052253,-76.161910,nan
1,W:\2019\20191206_001_Iver3069_ABS1,20191206_001_Iver3069_ABS1,20191206,1,Iver3069,ABS1,44.047112,-76.194706,44.055626,-76.184087,nan
2,W:\2019\20191206_002_Iver3069_ABS1,20191206_002_Iver3069_ABS1,20191206,2,Iver3069,ABS1,44.045722,-76.193327,44.053401,-76.182277,nan
3,M:\2019\20190410_001_Iver3069_ABS1,20190410_001_Iver3069_ABS1,20190410,1,Iver3069,ABS1,NaN,NaN,NaN,NaN,nan
4,M:\2019\20190425_001_Iver3069_ABS1,20190425_001_Iver3069_ABS1,20190425,1,Iver3069,ABS1,NaN,NaN,NaN,NaN,nan
...,...,...,...,...,...,...,...,...,...,...,...
1366,X:\AUV Collects\2023\3098\20230730_001_Iver309...,20230730_001_Iver3098_ABS2,20230730,1,Iver3098,ABS2,42.421591,-86.402124,42.445038,-86.280032,nan
1367,X:\AUV Collects\2023\3098\20230731_001_Iver309...,20230731_001_Iver3098_ABS2,20230731,1,Iver3098,ABS2,41.691396,-87.009579,41.785677,-86.988987,nan
1368,X:\AUV Collects\2023\3098\20230801_001_Iver309...,20230801_001_Iver3098_ABS2,20230801,1,Iver3098,ABS2,41.622285,-87.275083,41.716532,-87.272514,nan
1369,X:\AUV Collects\2023\3098\20230802_001_Iver309...,20230802_001_Iver3098_ABS2,20230802,1,Iver3098,ABS2,42.560323,-87.807816,42.569292,-87.679093,nan


In [6]:
BayHarbor = df_tiffs.loc[0]
ElkRapids = df_tiffs.loc[1]
FishermansIsland = df_tiffs.loc[2]
IngallsPoint = df_tiffs.loc[3]
MissionPeninsula = df_tiffs.loc[4]
TanneryCreek = df_tiffs.loc[5]
TraverseShoal = df_tiffs.loc[6]

In [8]:
def reef_overlap(reef, df):
    idx = df[(reef.avr_lat > df.min_lat) & (reef.avr_lat < df.max_lat) & (reef.avr_lon > df.min_lon) & (reef.avr_lon < df.max_lon)].index
    new_df = df.iloc[idx,:]
    new_df.loc[:,'Reef_overlap'] = reef.filename
    return new_df

BayHarbor_df = reef_overlap(BayHarbor, df)
ElkRapids_df = reef_overlap(ElkRapids, df)
FishermansIsland_df = reef_overlap(FishermansIsland, df)
IngallsPoint_df = reef_overlap(IngallsPoint, df)
MissionPeninsula_df = reef_overlap(MissionPeninsula, df)
TanneryCreek_df = reef_overlap(TanneryCreek, df)
TraverseShoal_df = reef_overlap(TraverseShoal, df)

dfs = [BayHarbor_df, ElkRapids_df, FishermansIsland_df, IngallsPoint_df, MissionPeninsula_df, TanneryCreek_df, TraverseShoal_df]
reef_overlap_df = pd.concat(dfs)
reef_overlap_df.sort_values(by=["Reef_overlap", "date"])
# reef_overlap.to_csv("Reef_tif_collects_overlap.csv")
reef_overlap_df

,collect_path,collect_id,date,collect,Iver_num,cam_sys,min_lat,min_lon,max_lat,max_lon,Reef_overlap
243,W:\2021\20210607_001_Iver3069_ABS1,20210607_001_Iver3069_ABS1,20210607,1,Iver3069,ABS1,45.371176,-84.996942,45.376671,-84.980733,BayHarbor_Classified2.tif
222,W:\2021\20210501_001_Iver3069_ABS1,20210501_001_Iver3069_ABS1,20210501,1,Iver3069,ABS1,44.904994,-85.420864,44.908793,-85.415701,ElkRapids_Classified2.tif
225,W:\2021\20210502_002_Iver3069_ABS1,20210502_002_Iver3069_ABS1,20210502,2,Iver3069,ABS1,44.898632,-85.429318,44.918893,-85.410185,ElkRapids_Classified2.tif
30,W:\2020\20200809_001_Iver3069_ABS1,20200809_001_Iver3069_ABS1,20200809,1,Iver3069,ABS1,45.282678,-85.375514,45.326692,-85.292970,FishermensIsland_Classified.tif
236,W:\2021\20210602_001_Iver3069_ABS1,20210602_001_Iver3069_ABS1,20210602,1,Iver3069,ABS1,45.289728,-85.369198,45.298197,-85.357964,FishermensIsland_Classified.tif
238,W:\2021\20210604_001_Iver3069_ABS1,20210604_001_Iver3069_ABS1,20210604,1,Iver3069,ABS1,45.292312,-85.362628,45.302572,-85.350665,FishermensIsland_Classified.tif
273,W:\2021\20210808_001_Iver3069_ABS1,20210808_001_Iver3069_ABS1,20210808,1,Iver3069,ABS1,45.276977,-85.390865,45.309481,-85.327671,FishermensIsland_Classified.tif
69,W:\2020\20200923_001_Iver3069_ABS1,20200923_001_Iver3069_ABS1,20200923,1,Iver3069,ABS1,45.066003,-85.575252,45.085739,-85.565549,IngallsPoint_Classified.tif
71,W:\2020\20200923_003_Iver3069_ABS1,20200923_003_Iver3069_ABS1,20200923,3,Iver3069,ABS1,45.066077,-85.575214,45.085694,-85.565381,IngallsPoint_Classified.tif
240,W:\2021\20210605_001_Iver3069_ABS1,20210605_001_Iver3069_ABS1,20210605,1,Iver3069,ABS1,45.069184,-85.575436,45.075498,-85.566900,IngallsPoint_Classified.tif


In [9]:
df_unp = pd.read_csv('all_unpacked_images.csv', index_col=0, low_memory=False)
df_unp.head()

,image_name,Heading_D,Pitch_D,Roll_D,Temp_C,AUV_depth_m,Alt_m,Salinity_ppt,Speed_kn,Lat_DD,Long_DD,EpochTime,INS - Altitude,Speed_mps,Datetime,image_path,collect_id,AUV,cam_sys
0,CI_1567697832_646.png,185.0,-22.5,-3.2,28.6,3.35280,2.977591,0.0,2.17,44.982509,-85.828001,1.567698e+09,NaN,NaN,2019-09-05 11:37:12.646,Z:\__Organized_Directories_InProgress\2019_Unp...,NaN,Iver3069,ABS1
1,CI_1567697836_665.png,187.1,-2.3,-3.0,28.7,4.48056,2.138782,0.0,2.04,44.982469,-85.827998,1.567698e+09,NaN,NaN,2019-09-05 11:37:16.665,Z:\__Organized_Directories_InProgress\2019_Unp...,NaN,Iver3069,ABS1
2,CI_1567697840_692.png,187.2,5.0,-4.4,28.7,4.54152,2.196998,0.0,2.08,44.982432,-85.827996,1.567698e+09,NaN,NaN,2019-09-05 11:37:20.692,Z:\__Organized_Directories_InProgress\2019_Unp...,NaN,Iver3069,ABS1
3,CI_1567697841_328.png,188.2,2.9,-4.1,28.7,4.45008,2.294230,0.0,2.09,44.982423,-85.827996,1.567698e+09,NaN,NaN,2019-09-05 11:37:21.328,Z:\__Organized_Directories_InProgress\2019_Unp...,NaN,Iver3069,ABS1
4,CI_1567697845_348.png,191.5,-5.1,-5.1,28.6,4.26720,2.233270,0.0,1.92,44.982384,-85.827996,1.567698e+09,NaN,NaN,2019-09-05 11:37:25.348,Z:\__Organized_Directories_InProgress\2019_Unp...,NaN,Iver3069,ABS1


In [13]:
BayHarbor

filename    BayHarbor_Classified2.tif
min_lat                     45.370376
min_lon                    -85.000113
max_lat                     45.378667
max_lon                    -84.980203
avr_lat                     45.374522
avr_lon                    -84.990158
Name: 0, dtype: object

In [16]:
def reef_images_overlap(reef, df):
    idx = df[(reef.max_lat > df.Lat_DD) & (reef.min_lat < df.Lat_DD) & (reef.max_lon > df.Long_DD) & (reef.min_lon < df.Long_DD)]
    # new_df = df.iloc[idx,:].reset_index()
    # new_df.loc[:,'Reef_overlap'] = reef.filename
    return idx

reef_images_overlap(BayHarbor, df_unp)

,image_name,Heading_D,Pitch_D,Roll_D,Temp_C,AUV_depth_m,Alt_m,Salinity_ppt,Speed_kn,Lat_DD,Long_DD,EpochTime,INS - Altitude,Speed_mps,Datetime,image_path,collect_id,AUV,cam_sys
